In [3]:
import pandas as pd
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv('/kaggle/input/forcdtaaa/train.csv')
data.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
data = data.dropna()
data["text"] = data["title"] + data["abstract"]



def NLP_cleaning(text):
    text_corpus = []
    i=0
    for sent in tqdm(text, desc='Cleaning'):
        # print(i, end =" ")
        i+=1
        sent = re.sub('<[^>]*>', '', sent)
        sent = re.sub('[^a-zA-z0-9]', ' ', sent)
        sent = sent.lower()
        text_corpus.append(sent)

    return text_corpus


text = data.text.values.tolist()
text_corpus = NLP_cleaning(text)
data['text'] = text_corpus
data['title'] = NLP_cleaning(data.title.values.tolist())
data['author'] = NLP_cleaning(data.author.values.tolist())



# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# data['label_number'] = label_encoder.fit_transform(data['label'])


val_df = pd.read_csv('/kaggle/input/forcdtaaa/val.csv')
val_df.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
val_df = val_df.dropna()
# val_df['label_number'] = label_encoder.transform(val_df['label'])
val_df["text"] = val_df["title"] + val_df["abstract"]
val_df['title'] = NLP_cleaning(val_df.title.values.tolist())
val_df['author'] = NLP_cleaning(val_df.author.values.tolist())
val_df['abstract'] = NLP_cleaning(val_df.abstract.values.tolist())
val_df['text'] = NLP_cleaning(val_df.text.values.tolist())



!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Cleaning: 100%|██████████| 8648/8648 [00:00<00:00, 11873.25it/s]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
data = data[['text','label']]
data

,text,label
0,measurement of the z gamma b jet cross sect...,Physics
1,modelling systemic price cojumps with hawkes f...,Quantitative Finance
2,encoding large information structures in linea...,Machine Learning
3,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems
4,analysis of aster data for mapping bauxite ric...,Sociology
...,...,...
41534,an ontology based approach for curriculum mapp...,Computer Engineering
41535,recruiting project managers a comparative ana...,Sociology
41536,optimal stopping for l\ evy processes and affi...,Statistics and Probability
41537,against tachyophobiawe examine the possible ex...,Physics


In [6]:
val_df = val_df[['text','label']]
val_df

,text,label
0,observational properties of simulated galaxies...,External Galaxies
2,su 3 yang mills hamiltonian in the flux tube ...,Quantum Physics
3,suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics"
4,many neighborly spheresthe result of padrol as...,Discrete Mathematics and Combinatorics
5,physical properties of reb2 under high pressur...,Materials Science and Engineering
...,...,...
8895,spectral properties of ngc 4151 and the estima...,Physical Processes
8896,energy correlation of bottom quarks from decay...,Physics
8897,scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences
8898,resisting structural re identification in anon...,Earth Sciences


In [7]:
X_train = data['text']
X_train = X_train.to_list()
Y_train = data['label_number']
Y_train = Y_train.to_list()

KeyError: 'label_number'

In [10]:
X_test = val_df['text'].to_list()
Y_test = val_df['label_number'].to_list()

In [11]:
train_embeddings = model.encode(X_train)
test_embeddings = model.encode(X_test)

Batches:   0%|          | 0/1261 [00:00<?, ?it/s]

Batches:   0%|          | 0/271 [00:00<?, ?it/s]

In [12]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score

svm_classifier = SVC(C=1.5, kernel='poly')
ovr_classifier = OneVsRestClassifier(svm_classifier)
ovr_classifier.fit(train_embeddings, Y_train)

y_pred = ovr_classifier.predict(test_embeddings)
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)
print("\n-----------------------------------------------\n\n")
print(classification_report(Y_test, y_pred))

Accuracy: 0.72895467160037

-----------------------------------------------


              precision    recall  f1-score   support

           0       0.75      0.70      0.72        99
           1       0.81      0.79      0.80       126
           2       0.65      0.70      0.67       130
           3       0.86      1.00      0.92        12
           4       0.67      0.57      0.61       118
           5       0.30      0.10      0.15        30
           6       0.65      0.53      0.58       104
           7       0.83      0.56      0.67         9
           8       0.58      0.66      0.62       557
           9       0.50      0.09      0.15        11
          10       0.71      0.69      0.70       225
          11       0.00      0.00      0.00         3
          12       0.77      0.92      0.84       189
          13       0.81      0.40      0.54        42
          14       0.00      0.00      0.00         3
          15       0.67      0.67      0.67         9
   

In [13]:
import pickle
pickle.dump(ovr_classifier, open('ovr_model.pkl', 'wb'))

In [9]:
test_df = pd.read_csv("/kaggle/input/testingfinal/testing.csv")
test_df

,abstract,author,doi,url,publication month,publication year,title,publisher,data_index
0,the paper introduces the notion of a locally c...,Bartosz Zielinski,NaN,NaN,NaN,NaN,Locally coalgebra-Galois extensions,NaN,22901
1,a bipartite state which is secretly chosen fro...,"Yangjia Li, Runyao Duan, and Mingsheng Ying",10.1103/PhysRevA.82.032339,NaN,NaN,NaN,Local Unambiguous Discrimination with Remainin...,NaN,47248
2,this short paper addresses the open problems l...,F\'elix Bou and Carles Noguera,NaN,NaN,NaN,NaN,Solution of some problems in the arithmetical ...,NaN,27601
3,we find the symmetry algebras of cosets which ...,"Dushyant Kumar, Menika Sharma",10.1007/JHEP08(2019)179,NaN,NaN,NaN,Symmetry Algebras of Stringy Cosets,NaN,36511
4,we present the results of the analysis of the ...,"E. Jimenez-Bailon (Universita Roma Tre), M. Sa...",10.1051/0004-6361:20065566,NaN,NaN,NaN,XMM-Newton view of the double-peaked Fe K-alph...,NaN,15483
...,...,...,...,...,...,...,...,...,...
8898,background studies show that the u.s. foreign-...,"['James W. Vaupel', 'Danan Gu', 'Matthew E. Du...",10.1371/journal.pone.0037177,NaN,5.0,2012.0,survival differences among native-born and for...,PLoS ONE,43679
8899,we prove uniqueness for the vortex-wave system...,"Christophe Lacave (ICJ), Evelyne Miot (LJLL)",NaN,NaN,NaN,NaN,Uniqueness for the vortex-wave system when the...,NaN,41425
8900,we demonstrate sensitivity of the mn 3d valenc...,"K. W. Edmonds, G. van der Laan, N. R. S. Farle...",10.1103/PhysRevB.77.113205,NaN,NaN,NaN,Strain dependence of the Mn anisotropy in ferr...,NaN,46483
8901,the coexisting regime of spin density wave (sd...,"Q. Q. Ge, Z. R. Ye, M. Xu, Y. Zhang, J. Jiang,...",10.1103/PhysRevX.3.011020,NaN,NaN,NaN,Anisotropic but nodeless superconducting gap i...,"Phys. Rev. X 3, 011020 (2013)",53461


In [10]:
test_df.isnull().sum()

abstract              255
author                 27
doi                  3712
url                  8498
publication month    7499
publication year     7496
title                   0
publisher            5148
data_index              0
dtype: int64

In [11]:
test_df.drop(['doi','url','publication month', 'publication year','publisher', 'author'], axis =1 , inplace = True)
test_df.fillna('', inplace=True)
test_df['text'] = test_df['title'] + test_df['abstract']
test_df['text'] = NLP_cleaning(test_df.text.values.tolist())
test_df = test_df[['text', 'data_index']]
test_df

Cleaning: 100%|██████████| 8903/8903 [00:00<00:00, 12072.22it/s]


,text,data_index
0,locally coalgebra galois extensionsthe paper i...,22901
1,local unambiguous discrimination with remainin...,47248
2,solution of some problems in the arithmetical ...,27601
3,symmetry algebras of stringy cosetswe find the...,36511
4,xmm newton view of the double peaked fe k alph...,15483
...,...,...
8898,survival differences among native born and for...,43679
8899,uniqueness for the vortex wave system when the...,41425
8900,strain dependence of the mn anisotropy in ferr...,46483
8901,anisotropic but nodeless superconducting gap i...,53461


In [12]:
model = SentenceTransformer('all-MiniLM-L6-v2')
ip = test_df['text'].tolist()
ip = model.encode(ip)

Batches:   0%|          | 0/279 [00:00<?, ?it/s]

In [21]:
preds = ovr_classifier.predict(ip)
preds

array([  0, 107,  66, ...,  68,  73, 115])

In [22]:
preds = label_encoder.inverse_transform(preds)

In [23]:
preds

array(['Algebra', 'Quantum Physics', 'Logic and Foundations', ...,
       'Materials Science and Engineering', 'Mechanics of Materials',
       'Software Engineering'], dtype=object)

In [24]:
submission = pd.DataFrame()
submission['data_index'] = test_df['data_index']
submission['target'] = preds
submission

,data_index,target
0,22901,Algebra
1,47248,Quantum Physics
2,27601,Logic and Foundations
3,36511,Quantum Physics
4,15483,Astrophysics and Astronomy
...,...,...
8898,43679,Bioinformatics
8899,41425,Dynamics/Dynamical Systems
8900,46483,Materials Science and Engineering
8901,53461,Mechanics of Materials


In [25]:
import zipfile
submission.to_csv('predictions.csv')
zipfile.ZipFile('predictions.zip', mode='w').write("predictions.csv")

# Combined

In [13]:
data

,text,label
0,measurement of the z gamma b jet cross sect...,Physics
1,modelling systemic price cojumps with hawkes f...,Quantitative Finance
2,encoding large information structures in linea...,Machine Learning
3,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems
4,analysis of aster data for mapping bauxite ric...,Sociology
...,...,...
41534,an ontology based approach for curriculum mapp...,Computer Engineering
41535,recruiting project managers a comparative ana...,Sociology
41536,optimal stopping for l\ evy processes and affi...,Statistics and Probability
41537,against tachyophobiawe examine the possible ex...,Physics


In [14]:
val_df

,text,label
0,observational properties of simulated galaxies...,External Galaxies
2,su 3 yang mills hamiltonian in the flux tube ...,Quantum Physics
3,suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics"
4,many neighborly spheresthe result of padrol as...,Discrete Mathematics and Combinatorics
5,physical properties of reb2 under high pressur...,Materials Science and Engineering
...,...,...
8895,spectral properties of ngc 4151 and the estima...,Physical Processes
8896,energy correlation of bottom quarks from decay...,Physics
8897,scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences
8898,resisting structural re identification in anon...,Earth Sciences


In [15]:
comb = pd.concat([data,val_df])
comb

,text,label
0,measurement of the z gamma b jet cross sect...,Physics
1,modelling systemic price cojumps with hawkes f...,Quantitative Finance
2,encoding large information structures in linea...,Machine Learning
3,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems
4,analysis of aster data for mapping bauxite ric...,Sociology
...,...,...
8895,spectral properties of ngc 4151 and the estima...,Physical Processes
8896,energy correlation of bottom quarks from decay...,Physics
8897,scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences
8898,resisting structural re identification in anon...,Earth Sciences


In [16]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
comb['label_number'] = label_encoder.fit_transform(comb['label'])

In [17]:
comb

,text,label,label_number
0,measurement of the z gamma b jet cross sect...,Physics,95
1,modelling systemic price cojumps with hawkes f...,Quantitative Finance,105
2,encoding large information structures in linea...,Machine Learning,67
3,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems,41
4,analysis of aster data for mapping bauxite ric...,Sociology,114
...,...,...,...
8895,spectral properties of ngc 4151 and the estima...,Physical Processes,93
8896,energy correlation of bottom quarks from decay...,Physics,95
8897,scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences,84
8898,resisting structural re identification in anon...,Earth Sciences,42


In [18]:
X = comb['text']
X = X.to_list()
Y = comb['label_number']
Y = Y.to_list()

In [19]:
train_emb = model.encode(X)

Batches:   0%|          | 0/1531 [00:00<?, ?it/s]

In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score

svm_classifier = SVC(C=1.5, kernel='poly')
ovr = OneVsRestClassifier(svm_classifier)
ovr.fit(train_emb, Y)

In [ ]:
preds1 = ovr.predict(ip)
preds1 = label_encoder.inverse_transform(preds)
submission = pd.DataFrame()
submission['data_index'] = test_df['data_index']
submission['target'] = preds1
submission.to_csv('predictions1.csv')
zipfile.ZipFile('predictions1.zip', mode='w').write("predictions1.csv")